In [1]:
import string
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
mystem = Mystem()
morph = MorphAnalyzer()

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Александра\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Александра\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline  
pd.set_option('max_colwidth', 1000)

In [4]:
train_data = pd.read_csv('data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

In [5]:
train_data.head()

,label,text
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн"
1,0,«Мегафон» стал владельцем 50% акций «Евросети»
2,-1,"RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …"
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0
4,-1,"@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!"


In [6]:
test_data.head()

,label,text
0,-1,RT vzglyad: По делу о работе МТС в Узбекистане США предложили заморозить 300 млн
1,0,RT @kevinuyatukox: http://t.co/ljtrjq91v3 #Кредитные карты мегафон банка
2,0,#Оформить кредитную карту в банке мтс http://t.co/vv1B6PMWgH
3,0,#Как перевести деньги с билайна на кредитную карту
4,0,#Начальник отдела кредитного контроля оао мтс усачева н а


## Задание 1.0
Бейзлайн без нормализации

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
def Vecrotizer(train_data, test_data):
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(train_data.values) 

    X_train = count_vectorizer.transform(train_data.values)
    X_test = count_vectorizer.transform(test_data.values)
    return X_train, X_test, count_vectorizer

In [9]:
X_train, X_test, count_vectorizer = Vecrotizer(train_data.text, test_data.text)

In [10]:
X_train.shape

(8208, 20511)

In [11]:
X_test.shape

(2054, 20511)

In [12]:
y_train = train_data.label.values
y_test = test_data.label.values

In [13]:
def gridS(X_train, y_train):
    log = LogisticRegression(penalty="l1")
    params = {'C': [0.01, 0.1, 1, 10, 100]}
    clf = GridSearchCV(log, param_grid=params)
    clf.fit(X_train, y_train)
    return clf

In [14]:
clf = gridS(X_train, y_train)

In [15]:
clf.best_params_

{'C': 10}

In [16]:
def logistic(X_train_2, y_train, X_test_2, y_test):
    clf = LogisticRegression(penalty="l1", C=10)
    clf.fit(X_train_2, y_train)
    y_pred_2 = clf.predict(X_test_2)
    normal = accuracy_score(y_test, y_pred_2)
    return clf, normal, y_pred_2

In [17]:
clf, base_line, y_pred = logistic(X_train, y_train, X_test, y_test)

In [18]:
base_line

0.6718597857838364

In [19]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.66      0.70       902
          0       0.66      0.76      0.71       972
          1       0.35      0.26      0.30       180

avg / total       0.67      0.67      0.67      2054

Макросредняя F1 мера -  0.568121798443427
Микросредняя F1 мера -  0.6718597857838364


In [20]:
def significant(vectorizer, clf, top=10):
    words = vectorizer.get_feature_names()
    classes = clf.classes_
    imp = clf.coef_
    for el, cls in enumerate(classes):
        sign_words = sorted(list(zip(words, imp[el])), key=lambda x: abs(x[1]), reverse=True)[:top]
        print('Class ' + str(cls) + ': \n',
              [word for word,_ in sign_words], '\n\n'
             )


In [21]:
significant(count_vectorizer, clf, top=10)

Class -1: 
 ['собираются', 'сбой', 'evakobb', 'обман', 'отправлялись', 'нему', 'керчи', 'перебоями', 'amaranth815', 'прогресс'] 


Class 0: 
 ['нему', 'собираются', 'хороший', 'жителям', 'прокомментировала', 'jivh4eenpq', 'расторгнуть', 'иа', 'прочность', 'гавно'] 


Class 1: 
 ['здорово', 'фотоед', 'специалистом', 'ожидал', 'радовать', 'обожаю', 't1cw25qbrz', 'расширяется', 'люблю', 'выросла'] 




Попробуем улучшить результат с помощью лемматицации

## Задание 1.1
Лемматизация

In [22]:
from string import punctuation, digits

Готовый набор стоп-слов для русского языка возьмём из nltk

In [23]:
stop_words = stopwords.words('russian')
stop_words[:6]

['и', 'в', 'во', 'не', 'что', 'он']

In [24]:
punctuation = set(punctuation + '«»—…“”\n\t' + digits)
punctuation.remove('@')
table = str.maketrans({ch: None for ch in punctuation})

In [25]:
def normalize(text):
    
    clean_data = []
    
    good_words = [
        word for word in word_tokenize(text) if len(text) > 1 and not all([ch in punctuation for ch in word])]
    
    words_normalized = [morph.parse(token)[0].normal_form for token in good_words]
    
    for word in words_normalized:
        if word not in stop_words:
            clean_data.append(word)

    return ' '.join(clean_data)

In [26]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

In [27]:
X_train_2, X_test_2, count_vectorizer = Vecrotizer(train_data.normalized, test_data.normalized)

In [28]:
X_train_2.shape

(8208, 14436)

In [29]:
X_test_2.shape

(2054, 14436)

In [30]:
clf, normalized, y_pred_2 = logistic(X_train_2, y_train, X_test_2, y_test)

In [31]:
normalized

0.6640701071080818

In [32]:
print(classification_report(y_test, y_pred_2))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred_2, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred_2, average='micro'))

             precision    recall  f1-score   support

         -1       0.75      0.62      0.67       902
          0       0.66      0.77      0.71       972
          1       0.35      0.32      0.34       180

avg / total       0.67      0.66      0.66      2054

Макросредняя F1 мера -  0.5733628472525057
Микросредняя F1 мера -  0.6640701071080818


Результат немного ухудшился. Скорее всего, это связано с тем, что в рамках твита (длина которого ограничена 140 символами), некоторые стоп-слова имеют в реальности большое значение. Попробуем улучшить результат как-то по-другому

## Задания 2.0-2.2
Проанализировать важные признаки, fp, fn, confusion matrix, изменить правила препроцессинга

In [33]:
from sklearn.metrics import confusion_matrix

Для начала, попробуем не убирать стоп-слова

In [34]:
def normalize_stops(text):
    
    
    good_words = [
        word for word in word_tokenize(text) if len(text) > 1 and not all([ch in punctuation for ch in word])]
    
    words_normalized = [morph.parse(token)[0].normal_form for token in good_words]
    
    return ' '.join(words_normalized)

In [35]:
train_data['normalized_with_stops'] = train_data['text'].apply(normalize_stops)
test_data['normalized_with_stops'] = test_data['text'].apply(normalize_stops)

In [36]:
X_train_3, X_test_3, count_vectorizer = Vecrotizer(train_data.normalized_with_stops, test_data.normalized_with_stops)

In [37]:
clf, normalized_with_stops, y_pred_3 = logistic(X_train_3, y_train, X_test_3, y_test)

In [38]:
normalized_with_stops

0.6786757546251218

In [39]:
print(classification_report(y_test, y_pred_3))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred_3, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred_3, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       902
          0       0.67      0.76      0.71       972
          1       0.42      0.33      0.37       180

avg / total       0.68      0.68      0.68      2054

Макросредняя F1 мера -  0.5935328585793745
Микросредняя F1 мера -  0.6786757546251218


In [40]:
significant(count_vectorizer, clf, top=10)

Class -1: 
 ['задолженность', 'amaranth815', 'оштрафовать', 'атаковать', 'сбой', 'турбокнопка', 'уезжать', 'добиться', 'расторгнуть', 'испытывать'] 


Class 0: 
 ['650р', 'топливо', 'расторгнуть', 'гавный', 'испытывать', 'вносить', 'достоверно', 'слогана', 'инноватор', 'поведенческий'] 


Class 1: 
 ['lizinastusha', 'здорово', 'адекватный', '6j3mfzcy5u', 'мило', 'топливо', 'расширяться', 'инноватор', 'youdicuudv', 'защита'] 




Что получилось:
- некоторые слова относятся к нескольким классам ("топливо", "инноватор", "испытывать", "расторгнуть"), что усложняет их разделение
- класс с положительными твитами может отделяется значительно хуже остальных. Думаю, это связано с тем, что идентефицировать негативно окрашенные слова проще, чем положительно. 

Проверим последний пункт по Confusion matrix

In [41]:
confusion_matrix(y_test, y_pred_3)

array([[600, 272,  30],
       [186, 734,  52],
       [ 37,  83,  60]], dtype=int64)

Действительно, наша программа плохо разграничивает классы

## Задание 2.3-2.4
Подобрать параметры в классификаторе и векторайзере и произвести отбор признаков

Сначала сравним те модели, которые у нас уже есть

In [42]:
def TfidfVectorize(train_data, test_data):
    tfidf = TfidfVectorizer()
    tfidf.fit(train_data.values)
    X_train = tfidf.transform(train_data.values)
    X_test = tfidf.transform(test_data.values)
    return X_train, X_test, tfidf

In [43]:
X_train, X_test, tfidf = TfidfVectorize(train_data.text, test_data.text)
clf, base_line_tf, y_pred = logistic(X_train, y_train, X_test, y_test)

In [44]:
X_train_2, X_test_2, tfidf_2 = TfidfVectorize(train_data.normalized, test_data.normalized)
clf, normalized_stop_tf, y_pred_2 = logistic(X_train_2, y_train, X_test_2, y_test)

In [45]:
X_train_3, X_test_3, tfidf_3 = TfidfVectorize(train_data.normalized_with_stops, test_data.normalized_with_stops)
clf, normalized_no_stops_tf, y_pred_3 = logistic(X_train_3, y_train, X_test_3, y_test)

In [46]:
base_line_tf

0.6650438169425511

In [47]:
normalized_stop_tf

0.6504381694255112

In [48]:
normalized_no_stops_tf

0.6713729308666018

In [49]:
significant(tfidf, clf, top=10)

Class -1: 
 ['кобо', 'нахрен', 'sashaselfish', 'zi24akrlsw', 'быстрые', 'adoralim', 'маршрутизатора', 'инет', 'отдых', 'грамотности'] 


Class 0: 
 ['бреста', 'оповещениями', 'гейнеры', 'биатлону', 'набрехали', '3гб', 'краже', 'внятно', 'перебежчиках', 'голос'] 


Class 1: 
 ['kalars07', 'y6atjvkiud', 'вышки', 'лицензионным', 'загрузились', 'выставка', 'голос', '424', 'перезвонит', 'влияет'] 




In [50]:
significant(tfidf_2, clf, top=10)

Class -1: 
 ['памятник', 'светлый', 'tele2', 'атаковать', 'говорить', 'beeline_omsk', 'прибытие', 'ободрать', 'україный', 'исходящи'] 


Class 0: 
 ['газ', 'транзит', 'игра', 'восточный', 'реагировать', '650р', 'повторяться', 'домен', 'ция', 'инстаграмм'] 


Class 1: 
 ['lizinastusha', 'адекватный', 'зеленодольск', 'попробовать', 'мтс970', 'збс', 'инстаграмм', '6j3mfzcy5u', 'честнаялига', 'договариваться'] 




In [51]:
significant(tfidf_3, clf, top=10)

Class -1: 
 ['оштрафовать', 'сбой', 'tele2', 'атаковать', 'говно', 'beeline_omsk', 'прекрасно', 'обман', 'уезжать', 'испытывать'] 


Class 0: 
 ['гавный', 'топливо', 'иа', 'восстановление', 'расторгнуть', '650р', 'поведенческий', 'доллар', 'хуй', 'инноватор'] 


Class 1: 
 ['lizinastusha', 'адекватный', 'здорово', 'понравиться', 'мощь', 'защита', 'инноватор', '6j3mfzcy5u', 'частный', 'довольный'] 




In [52]:
pd.DataFrame({'CountVec': [base_line, normalized, normalized_with_stops],
              'TfidfVec': [base_line_tf, normalized_stop_tf, normalized_no_stops_tf]
             }, index=['StarterPack', 'Normalized_no_Stops', 'Normalized_with_Stops'],)

,CountVec,TfidfVec
StarterPack,0.671860,0.665044
Normalized_no_Stops,0.664070,0.650438
Normalized_with_Stops,0.678676,0.671373


C tf-idf стало хуже, постараемся улучшить CountVectorizer

Перейдем к отбору признаков:
- создадим свой список стоп-слов, т.е. слов, которые присутсвуют во всех классах, а значит, не играют большой роли при классификации

In [53]:
def common(vectorizer, clf, top=10):
    words = vectorizer.get_feature_names()
    classes = clf.classes_
    imp = clf.coef_
    result = []
    for el, cls in enumerate(classes):
        important_words = sorted(list(zip(words, imp[el])), key=lambda x: abs(x[1]), reverse=True)[len(imp) - top:]
        result.append(set(important_words))
    return result

In [54]:
common_data = common(count_vectorizer, clf, top=2400)

In [55]:
#common_data

In [56]:
data_un = common_data[0] & common_data[1] & common_data[2]
stop_words = []
for word in data_un:
    stop_words.append(word[0])

In [57]:
len(stop_words)

1745

In [58]:
#stop_words_2

Посмотрим, как это сработает

In [59]:
train_data['normalized_new_stops'] = train_data['text'].apply(normalize)
test_data['normalized_new_stops'] = test_data['text'].apply(normalize)

In [60]:
X_train_4, X_test_4, count_vectorizer = Vecrotizer(train_data.normalized_new_stops,
                                                 test_data.normalized_new_stops)
clf, normalized_new_stops, y_pred_4 = logistic(X_train_4, y_train, X_test_4, y_test)
normalized_new_stops

0.6806231742940604

In [61]:
print(classification_report(y_test, y_pred_4))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred_4, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred_4, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       902
          0       0.68      0.76      0.71       972
          1       0.43      0.33      0.37       180

avg / total       0.68      0.68      0.68      2054

Макросредняя F1 мера -  0.5952340216744538
Микросредняя F1 мера -  0.6806231742940604
